In [3]:
import numpy as np
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification

In [27]:
# Load pre-trained model (weights)
model_version = '/virtualmachines/models/bert_finetuned_cola_bs64'
#model_version = '/virtualmachines/models/bert_multilingual_finetuned_6epochs'
model = BertForSequenceClassification.from_pretrained(model_version,num_labels=2)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda(0)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            if len(new_sent) == 0:
                continue
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

In [28]:
cuda = True
batch = tokenize_batch([["[CLS]", "the", "dog", "see", "i", ".", "[SEP]"]])
print(batch)
inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
model(inp)


[[101, 1103, 3676, 1267, 178, 119, 102]]


tensor([[-0.5546,  1.5437]], device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
import csv
kaggle = csv.reader(open("/virtualmachines/data/quora/train.csv","r"), delimiter=",")
with open("/virtualmachines/data/quora/scrambled/train.csv") as out_train:
    with open("/virtualmachines/data/quora/scrambled/train.csv") as out_test:
        for line in kaggle:
            s1 = line[3]
            s2 = line[4]
            
            if random.random() > 0.33:
                out_train.write("1",)
    